# Trajectory Optimization

This jupyter notebook is the main trajectory optimization notebook used in Van Tran's thesis.

## Main Run

In [1]:
import openap
from openap import top
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Outage and Risk Objective Function Setup

In [2]:
# Load the CSV data into a Pandas DataFrame
csv_file_path = "data/cost/40_outage_data.csv"  # Replace with your file path
outage_likelihood_data = pd.read_csv(csv_file_path)

# Extract unique values of latitude, longitude, and altitude
latitudes = outage_likelihood_data['Latitude'].unique()
longitudes = outage_likelihood_data['Longitude'].unique()
altitudes = outage_likelihood_data['Altitude'].unique()

# Sort unique values to ensure proper indexing
latitudes = np.sort(latitudes)
longitudes = np.sort(longitudes)
altitudes = np.sort(altitudes)

# Create a 3D matrix for outage likelihood values
outage_likelihood_values = np.full((len(latitudes), len(longitudes), len(altitudes)), np.nan)

# Populate the 3D matrix with outage probabilities
for _, row in outage_likelihood_data.iterrows():
    lat_idx = np.where(latitudes == row['Latitude'])[0][0]
    lon_idx = np.where(longitudes == row['Longitude'])[0][0]
    alt_idx = np.where(altitudes == row['Altitude'])[0][0]
    outage_likelihood_values[lat_idx, lon_idx, alt_idx] = row['Outage']

# Verify the shape of the resulting 3D matrix
print("Shape of outage likelihood matrix:", outage_likelihood_values.shape)

Shape of outage likelihood matrix: (351, 751, 31)


In [3]:
lons, lats, alts = np.meshgrid(longitudes, latitudes, altitudes)
likelihood_cost_3d = pd.DataFrame(
    np.array([lons, lats, alts, outage_likelihood_values]).reshape(4, -1).T,
    columns=["longitude", "latitude", "altitude", "cost"],
).assign(height=lambda x: x.altitude * 0.3048)

likelihood_cost_3d

,longitude,latitude,altitude,cost,height
0,-130.0,20.0,0.0,1.0,0.0
1,-130.0,20.0,1000.0,1.0,304.8
2,-130.0,20.0,2000.0,1.0,609.6
3,-130.0,20.0,3000.0,1.0,914.4
4,-130.0,20.0,4000.0,1.0,1219.2
...,...,...,...,...,...
8171626,-60.0,55.0,26000.0,1.0,7924.8
8171627,-60.0,55.0,27000.0,1.0,8229.6
8171628,-60.0,55.0,28000.0,1.0,8534.4
8171629,-60.0,55.0,29000.0,1.0,8839.2


In [4]:
# Load the CSV data into a Pandas DataFrame
csv_file_path = 'data/cost/40_risk_data.csv' # Replace with your file path
risk_data = pd.read_csv(csv_file_path)

# Extract unique values of latitude, longitude, and altitude
latitudes = risk_data['Latitude'].unique()
longitudes = risk_data['Longitude'].unique()
altitudes = risk_data['Altitude'].unique()

# Sort unique values to ensure proper indexing
latitudes = np.sort(latitudes)
longitudes = np.sort(longitudes)
altitudes = np.sort(altitudes)

# Create a 3D matrix for outage likelihood values
risk_values = np.full((len(latitudes), len(longitudes), len(altitudes)), np.nan)

# Populate the 3D matrix with outage probabilities
for _, row in risk_data.iterrows():
    lat_idx = np.where(latitudes == row['Latitude'])[0][0]
    lon_idx = np.where(longitudes == row['Longitude'])[0][0]
    alt_idx = np.where(altitudes == row['Altitude'])[0][0]
    risk_values[lat_idx, lon_idx, alt_idx] = row['Risk']

# Verify the shape of the resulting 3D matrix
print("Shape of risk matrix:", risk_values.shape)

Shape of risk matrix: (351, 701, 31)


In [5]:
lons, lats, alts = np.meshgrid(longitudes, latitudes, altitudes)
risk_cost_3d = pd.DataFrame(
    np.array([lons, lats, alts, risk_values]).reshape(4, -1).T,
    columns=["longitude", "latitude", "altitude", "cost"],
).assign(height=lambda x: x.altitude * 0.3048)

risk_cost_3d

,longitude,latitude,altitude,cost,height
0,-130.0,20.0,0.0,0.0,0.0
1,-130.0,20.0,1000.0,0.0,304.8
2,-130.0,20.0,2000.0,0.0,609.6
3,-130.0,20.0,3000.0,0.0,914.4
4,-130.0,20.0,4000.0,0.0,1219.2
...,...,...,...,...,...
7627576,-60.0,55.0,26000.0,0.0,7924.8
7627577,-60.0,55.0,27000.0,0.0,8229.6
7627578,-60.0,55.0,28000.0,0.0,8534.4
7627579,-60.0,55.0,29000.0,0.0,8839.2


## Main Run for Flight Profile Extraction

In [6]:
import os

mass_takeoff_assumed = 66300  # Assumed takeoff fuel mass (kg)
fuelflow = openap.FuelFlow("A320") # Assumed aircraft

# Setting up objective functions
likelihood_interpolant = top.tools.interpolant_from_dataframe(likelihood_cost_3d)
risk_interpolant = top.tools.interpolant_from_dataframe(risk_cost_3d)

def objective(x, u, dt, **kwargs):
    """The final objective is the grid cost"""
    grid_cost = optimizer.obj_grid_cost(x, u, dt, time_dependent=True, **kwargs)
    return grid_cost

# Define your unique OD_pair_file
unique_OD_pair_file = 'data/misc/DFW_unique_OD_pairs.csv'

# Load the dataset
df = pd.read_csv(unique_OD_pair_file)

# Counters for total flights and successfully processed flights
total_flights = 0
successful_flights = 0

# Loop through rows in the file
for _, row in df.iterrows():
    # Increment flight count
    total_flights += 1

    # Extract origin and destination
    origin = row['Origin']
    destination = row['Destination']

    print(f"Processing flight {origin} to {destination}...")

    # Initialize optimizer for this flight
    optimizer = top.CompleteFlight("A320", origin, destination, m0=0.85)

    # Step 1: Optimize for risk
    risk_optimal_flight = optimizer.trajectory(objective=objective, interpolant=risk_interpolant)
    if risk_optimal_flight is None:
        print(f"Skipping Flight {origin} to {destination}: Risk optimization returned None.")
        continue

    # Step 2: Optimize for likelihood
    likelihood_optimal_flight = optimizer.trajectory(objective=objective, interpolant=likelihood_interpolant)
    if likelihood_optimal_flight is None:
        print(f"Skipping Flight {origin} to {destination}: Likelihood optimization returned None.")
        continue

    # Step 3: Optimize for fuel
    fuel_optimal_flight = optimizer.trajectory(objective="fuel")
    if fuel_optimal_flight is None:
        print(f"Skipping Flight {origin} to {destination}: Fuel optimization returned None.")
        continue

    # Increment successful flight count
    successful_flights += 1

    # Store results for each regime
    for regime, flight in {
        "risk": risk_optimal_flight,
        "likelihood": likelihood_optimal_flight,
        "fuel": fuel_optimal_flight,
    }.items():
        # Save each flight's trajectory to CSV
        output_folder = "data/flight_trajectories/40dB"
        os.makedirs(output_folder, exist_ok=True)  # Create the folder if it doesn't exist

        for regime, flight in {
            "risk": risk_optimal_flight,
            "likelihood": likelihood_optimal_flight,
            "fuel": fuel_optimal_flight,
        }.items():
            filename = f"{origin}_{destination}_{regime}_40_trajectory.csv"
            filepath = os.path.join(output_folder, filename)
            flight.to_csv(filepath, index=False)

# Calculate the percentage of successful flights
if total_flights > 0:
    success_rate = (successful_flights / total_flights) * 100
else:
    success_rate = 0

print(f"Processed {successful_flights}/{total_flights} flights successfully.")
print(f"Success Rate: {success_rate:.2f}%")

Processing flight KAFW to KMCI...


CasADi - 2025-03-26 18:42:15 WARNING("KeyboardInterruptException") [.../casadi/interfaces/ipopt/ipopt_nlp.cpp:114]


SystemError: <built-in function Function_call> returned a result with an exception set

## Main Run for Fuel Calculation

In [ ]:
mass_takeoff_assumed = 66300  # Assumed takeoff fuel mass (kg)
fuelflow = openap.FuelFlow("A320") # Assumed aircraft

# Setting up objective functions
likelihood_interpolant = top.tools.interpolant_from_dataframe(likelihood_cost_3d)
risk_interpolant = top.tools.interpolant_from_dataframe(risk_cost_3d)

def objective(x, u, dt, **kwargs):
    """The final objective is the grid cost"""
    grid_cost = optimizer.obj_grid_cost(x, u, dt, time_dependent=True, **kwargs)
    return grid_cost

# Define your unique OD_pair_file
unique_OD_pair_file = 'data/misc/DFW_unique_OD_pairs.csv'

# Load the dataset
df = pd.read_csv(unique_OD_pair_file)

# Lists to store total fuel for all regimes
fuel_burns_by_regime = {
    "fuel": [],
    "likelihood": [],
    "risk": []
}

time_in_flight_by_regime = {
    "fuel": [],
    "likelihood": [],
    "risk": []
}

# Counters for total flights and successfully processed flights
total_flights = 0
successful_flights = 0

# Loop through rows in the file
for _, row in df.iterrows():
    # Increment flight count
    total_flights += 1

    # Extract origin and destination
    origin = row['Origin']
    destination = row['Destination']

    print(f"Processing flight {origin} to {destination}...")

    # Initialize optimizer for this flight
    optimizer = top.CompleteFlight("A320", origin, destination, m0=0.85)

    # Step 1: Optimize for risk
    risk_optimal_flight = optimizer.trajectory(objective=objective, interpolant=risk_interpolant)
    if risk_optimal_flight is None:
        print(f"Skipping Flight {origin} to {destination}: Risk optimization returned None.")
        continue

    # Step 2: Optimize for likelihood
    likelihood_optimal_flight = optimizer.trajectory(objective=objective, interpolant=likelihood_interpolant)
    if likelihood_optimal_flight is None:
        print(f"Skipping Flight {origin} to {destination}: Likelihood optimization returned None.")
        continue

    # Step 3: Optimize for fuel
    fuel_optimal_flight = optimizer.trajectory(objective="fuel")
    if fuel_optimal_flight is None:
        print(f"Skipping Flight {origin} to {destination}: Fuel optimization returned None.")
        continue

    # Increment successful flight count
    successful_flights += 1

    # Store results for each regime
    for regime, flight in {
        "risk": risk_optimal_flight,
        "likelihood": likelihood_optimal_flight,
        "fuel": fuel_optimal_flight,
    }.items():
        # Ensure flight data exists
        if 'ts' not in flight.columns:
            print(f"Skipping {regime} optimization for {origin} to {destination}: No time series data found.")
            continue
            
        # Convert time from seconds to minutes
        total_time = max(flight['ts']) / 60  
        time_in_flight_by_regime[regime].append(total_time)

        # Compute time step differences
        flight = flight.assign(d_ts=flight['ts'].diff().fillna(0))

        # Initialize lists for fuel flow calculations
        fuelflow_every_step = []
        fuel_every_step = []
        mass_current = mass_takeoff_assumed  # Start with takeoff mass

        # Calculate fuel burn for the trajectory
        for _, row in flight.iterrows():
            ff = fuelflow.enroute(
                mass=mass_current,
                tas=row.tas,
                alt=row.altitude,
                vs=row.vertical_rate,
            )
            fuel = ff * row.d_ts
            fuelflow_every_step.append(ff)
            fuel_every_step.append(fuel)
            mass_current -= fuel

        # Calculate total fuel burn
        total_fuel = sum(fuel_every_step)
        fuel_burns_by_regime[regime].append(total_fuel)
    
    if total_flights == 3:
        break

# Calculate the percentage of successful flights
if total_flights > 0:
    success_rate = (successful_flights / total_flights) * 100
else:
    success_rate = 0

print(f"Processed {successful_flights}/{total_flights} flights successfully.")
print(f"Success Rate: {success_rate:.2f}%")

Processing flight KAFW to KMCI...
Skipping Flight KAFW to KMCI: Risk optimization returned None.
Processing flight KDFW to KAFW...
Skipping Flight KDFW to KAFW: Risk optimization returned None.
Processing flight KAFW to KMFE...
Skipping Flight KAFW to KMFE: Risk optimization returned None.
Processing flight KDFW to KGSO...
Skipping Flight KDFW to KGSO: Risk optimization returned None.
Processing flight KAFW to KCVG...
Skipping Flight KAFW to KCVG: Risk optimization returned None.
Processing flight KAFW to KMIA...
Skipping Flight KAFW to KMIA: Risk optimization returned None.
Processing flight KAFW to KMSY...
Processing flight KAFW to MROC...
Skipping Flight KAFW to MROC: Risk optimization returned None.
Processing flight KAFW to KIDA...
Skipping Flight KAFW to KIDA: Risk optimization returned None.
Processing flight KDFW to KPDX...
Skipping Flight KDFW to KPDX: Risk optimization returned None.
Processing flight KAFW to KJFK...
Skipping Flight KAFW to KJFK: Risk optimization returned No

CasADi - 2025-03-06 07:46:47 WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 5, col 0).") [.../casadi/core/oracle_function.cpp:393]


Skipping Flight KDFW to RKSI: Risk optimization returned None.
Processing flight KAFW to KDSM...
Skipping Flight KAFW to KDSM: Fuel optimization returned None.
Processing flight KAFW to KABE...
Skipping Flight KAFW to KABE: Risk optimization returned None.
Processing flight KAFW to KOKC...
Skipping Flight KAFW to KOKC: Likelihood optimization returned None.
Processing flight KAFW to KBDL...
Skipping Flight KAFW to KBDL: Risk optimization returned None.
Processing flight KAFW to KTOL...
Skipping Flight KAFW to KTOL: Risk optimization returned None.
Processing flight KAFW to KGEG...
Skipping Flight KAFW to KGEG: Risk optimization returned None.
Processing flight KAFW to KABQ...
Skipping Flight KAFW to KABQ: Fuel optimization returned None.
Processing flight KAFW to KEWR...
Skipping Flight KAFW to KEWR: Risk optimization returned None.
Processing flight KDAL to MMSP...
Skipping Flight KDAL to MMSP: Risk optimization returned None.
Processing flight KDFW to KTUL...
Skipping Flight KDFW to 

CasADi - 2025-03-06 10:27:00 WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 8, col 0).") [.../casadi/core/oracle_function.cpp:393]
CasADi - 2025-03-06 10:27:00 WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 8, col 0).") [.../casadi/core/oracle_function.cpp:393]


Skipping Flight KAFW to KAFW: Risk optimization returned None.
Processing flight KDAL to KYIP...
Skipping Flight KDAL to KYIP: Risk optimization returned None.
Processing flight KDAL to KMQY...
Skipping Flight KDAL to KMQY: Risk optimization returned None.
Processing flight KDAL to KMRC...
Skipping Flight KDAL to KMRC: Risk optimization returned None.
Processing flight KDAL to KSAT...
Skipping Flight KDAL to KSAT: Risk optimization returned None.
Processing flight KDFW to KJAX...
Processing flight KDFW to KMCO...
Skipping Flight KDFW to KMCO: Risk optimization returned None.
Processing flight KDFW to KPHX...
Processing flight KAFW to KPDX...
Skipping Flight KAFW to KPDX: Risk optimization returned None.
Processing flight KDFW to KSDF...
Skipping Flight KDFW to KSDF: Risk optimization returned None.
Processing flight KDFW to KSJC...
Processing flight KDFW to KSHV...
Skipping Flight KDFW to KSHV: Risk optimization returned None.
Processing flight KDFW to KDEN...
Skipping Flight KDFW to K

**Saving File**

In [ ]:
def save_flight_results(df, fuel_burns_by_regime, time_in_flight_by_regime, output_file):
    # Create a list to store results
    results_list = []

    # Loop through stored results and construct rows
    for index, row in df.iterrows():
        origin = row["Origin"]
        destination = row["Destination"]

        for regime in ["fuel", "likelihood", "risk"]:
            if regime in fuel_burns_by_regime and regime in time_in_flight_by_regime:
                if index < len(fuel_burns_by_regime[regime]) and index < len(time_in_flight_by_regime[regime]):
                    fuel_burn = fuel_burns_by_regime[regime][index]
                    time = time_in_flight_by_regime[regime][index]
                else:
                    fuel_burn = "NA"
                    time = "NA"
            else:
                fuel_burn = "NA"
                time = "NA"

            results_list.append([origin, destination, regime, fuel_burn, time])

    # Convert to DataFrame
    final_results_df = pd.DataFrame(results_list, columns=["origin", "destination", "regime", "fuel burn", "time"])

    # Save to CSV
    final_results_df.to_csv(output_file, index=False)
    
    print(f"Results saved to {output_file}")

# Define the output file path
output_file = "data/results/40dB/40_fuel_burn_results.csv"

# Call the function to save results
save_flight_results(df, fuel_burns_by_regime, time_in_flight_by_regime, output_file)

Results saved to /Users/vantran/Documents/thesis/code/50_flight_optimization_results_no_time.csv
